In [11]:
import sys
import time
import numpy as np
from itertools import chain
import preprocessing as pp, graphing as gp, dg_analysis as da, \
structure_discovery as sd, output as op


# Global variables
max_reads = 2000
min_overlap = 0.3

path = '/data/ihwang/data_rna/PARIS/rRNA/'
args_reads = path + 'reads/AMT_HeLa6_trim_nodup_starhs45SAligned_N_1rpn_l15p2_geometric_NGmin_sorted_noDGNG.sam'
args_ref_seq = path + 'gold_standards/hsrRNA.fa'
args_ref_genes = path + 'gold_standards/hsrRNA_gene.bed'
args_regions = ['hs45S']
args_genes = ['5.8S']


# Initialize reference dict analysis dict and parse reads
ref_dict = pp.get_reference_dict(args_ref_seq, args_ref_genes)
analysis_dict = pp.get_analysis_dict(ref_dict, args_regions, args_genes)
reads_dict = pp.parse_reads(args_reads, ref_dict)


# Analyze reads to obtain DGs
dg_ind = 0
ng_ind = 0
for region in analysis_dict:
    for gene in analysis_dict[region]:
        gene_ids = [
            read_id for (read_id, read_info) in reads_dict.items() 
            if (read_info[4] == region) & (read_info[5] == gene) 
            & (read_info[6] == gene)
        ]
        if len(gene_ids) > 1:
            if len(gene_ids) > max_reads:
                inds_samp = np.random.choice(
                    len(gene_ids), max_reads, replace=False
                )
                graph = gp.graph_reads(
                    [gene_ids[ind] for ind in inds_samp], 
                    reads_dict, t=min_overlap
                )
            else:
                graph = gp.graph_reads(
                    gene_ids, reads_dict, t=min_overlap
                )
            reads_dg_dict, dg_ind = gp.cluster_graph(graph, dg_ind)
            dg_reads_dict = da.get_preliminary_dgs(
                reads_dict, reads_dg_dict
            )
            if len(gene_ids) > max_reads:
                inds_unsamp = np.setdiff1d(
                    range(len(gene_ids)), inds_samp
                )
                dg_reads_dict, dg_index = da.adjust_dgs(
                    dg_reads_dict, [
                        gene_ids[ind] for ind in inds_unsamp
                    ], 
                    reads_dict, dg_ind, t=min_overlap
                )
            dg_filtered_dict = da.filter_dgs(dg_reads_dict)

In [25]:
import itertools
list(itertools.chain.from_iterable(dg_reads_dict.values()))

['NS500615:86:HFM52BGXX:4:23610:11955:16526',
 'NS500615:86:HFM52BGXX:1:22203:5259:11343',
 'NS500615:86:HFM52BGXX:1:23104:15723:20085',
 'NS500615:86:HFM52BGXX:2:11302:1436:15988',
 'NS500615:86:HFM52BGXX:1:11210:14721:11524',
 'NS500615:86:HFM52BGXX:1:12306:16090:11134',
 'NS500615:86:HFM52BGXX:1:13102:13155:17955',
 'NS500615:86:HFM52BGXX:1:13208:1450:6978',
 'NS500615:86:HFM52BGXX:1:13302:17375:14572',
 'NS500615:86:HFM52BGXX:1:13303:13888:4907',
 'NS500615:86:HFM52BGXX:1:22307:20922:9519',
 'NS500615:86:HFM52BGXX:1:22311:2957:10857',
 'NS500615:86:HFM52BGXX:1:23204:11152:2866',
 'NS500615:86:HFM52BGXX:1:23303:13675:6810',
 'NS500615:86:HFM52BGXX:2:11106:21772:19527',
 'NS500615:86:HFM52BGXX:4:13502:14519:15242',
 'NS500615:86:HFM52BGXX:4:13601:21084:8330',
 'NS500615:86:HFM52BGXX:1:11101:9624:15569',
 'NS500615:86:HFM52BGXX:1:11106:9228:12827',
 'NS500615:86:HFM52BGXX:1:11108:5530:13753',
 'NS500615:86:HFM52BGXX:1:11109:13940:2242',
 'NS500615:86:HFM52BGXX:1:11109:8572:16399',
 'N